# Working with cloud masks

## Set up notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
from remote_sensing_tools.stac_configuration import de_australia_stac_config
from remote_sensing_tools.raster_base import RasterBase, QueryParams, LoadParams

## Set up and run query

### Get Digital Earth Australia configuration and list collections

In [ ]:
config = de_australia_stac_config

config.list_collections()

### Set query parameters and load Sentinel-2a and Sentinel-2b

In [ ]:
query_params = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2020-11-01",
    end_date="2020-12-01",
)

load_params = LoadParams(
    crs="EPSG:3577",
    resolution=10,
    bands=("red", "green", "blue", "nir", "fmask", "s2cloudless"),
)

stac_raster = RasterBase.from_stac_query(
    config=config,
    collections=["ga_s2am_ard_3", "ga_s2bm_ard_3"],
    query_params=query_params,
    load_params=load_params,
)

## Demonstrate masking

### Display an RGB plot for a subset of images

In [ ]:
stac_raster.data.isel(time=slice(0, 3))[
    ["red", "green", "blue"]
].to_array().plot.imshow(col="time", vmin=0, vmax=3000)

### Apply fmask and create new masked variables

In [ ]:
stac_raster.apply_mask("fmask", data_inplace=False, mask_inplace=False)

In [ ]:
stac_raster.data.isel(time=slice(0, 3))[
    ["red_masked", "green_masked", "blue_masked"]
].to_array().plot.imshow(col="time", vmin=0, vmax=3000)

#### Display new masked data variables

In [ ]:
stac_raster.data

#### Display new masks (converted categorical fmask to boolean)

In [ ]:
stac_raster.masks

### Apply s2cloudless and overwrite masked variables

In [ ]:
stac_raster.apply_mask("s2cloudless", data_inplace=False, mask_inplace=False)

In [ ]:
stac_raster.data.isel(time=slice(0, 3))[
    ["red_masked", "green_masked", "blue_masked"]
].to_array().plot.imshow(col="time", vmin=0, vmax=3000)

#### Display new masks (converted categorical s2cloudless to boolean)

In [ ]:
stac_raster.masks

### Demonstrate additional functionality: custom nodata and morphological operations

In [ ]:
import numpy as np

stac_raster.apply_mask(
    "fmask",
    mask_filters=[("opening", 3), ("dilation", 5)],
    nodata=np.nan,
    data_inplace=False,
    mask_inplace=False,
)

In [ ]:
stac_raster.data.isel(time=slice(0, 3))[
    ["red_masked", "green_masked", "blue_masked"]
].to_array().plot.imshow(col="time", vmin=0, vmax=3000)

#### Display new masks (boolean fmask with filtering applied)

In [ ]:
stac_raster.masks